In [31]:
import pandas as pd
from sklearn import linear_model
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
import matplotlib as mpl
import matplotlib.pyplot as plt
train = pd.read_csv("train_Madison.csv")
#train = train[:20]
test = pd.read_csv("test_Madison.csv")
names = train["name"]
stars = train["star"]
train.head(5)

,Id,star,name,city,postal_code,text,nchar,nword,gem,incredible,...,disappointing,manager,dirty,mediocre,poor,terrible,awful,rude,horrible,worst
0,39673,3,1847 At the Stamm House,Middleton,53562.0,"I see that 1847 is closing to ""retool,"" which ...",1964,352,0,0,...,0,0,0,0,1,0,0,0,0,0
1,36084,5,1847 At the Stamm House,Middleton,53562.0,"After a long, hard day at work, this was a won...",558,105,0,0,...,0,0,0,0,0,0,0,0,0,0
2,41867,5,1847 At the Stamm House,Middleton,53562.0,Really impressed. I had been here several yea...,549,88,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42318,4,1847 At the Stamm House,Middleton,53562.0,Good food and drink in a cool old house. Chees...,124,24,0,0,...,0,0,0,0,0,0,0,0,0,0
4,35080,5,1847 At the Stamm House,Middleton,53562.0,Love this place! Reminds me of Quivey's stable...,536,101,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
ratingTotal = {}
ratingCount = {}
for i in range(0, len(names)):
    name = names[i]
    star = stars[i]
    if name not in ratingTotal:
        ratingTotal[name] = 0
        ratingCount[name] = 0
    ratingTotal[name] += star
    ratingCount[name]+=1
print(len(ratingTotal))
print(len(ratingCount))
ratingAverage = dict((k, float(ratingTotal[k])/ratingCount[k]) for k in ratingCount)
#Turns the predictor from rating to difference above or below mean rating for the restaurant
#for i in range(0, len(names)):
#    name = names[i]
#    diffScore = stars[i]-ratingAverage[name]
#    print(diffScore)
#    diffScores.append(diffScore)

1150
1150


In [36]:
positiveCount = []
negativeCount = []
positiveOverall = []
#ratingArray = []
#nwords = []
posnshort = []
negnlong = []
nchars = []
#avgChars = []
#posnegratio= []
#numReviews = []
positiveWords = ["gem", "highly", "incredible", "amazing", "die", "favorites", "wonderful", "perfect", "fantastic", "notch",
                 "favorite", "awesome", "outstanding", "yum", "delicious", "excellent", "perfectly", "loved", "helpful", "best", 
                 "savory", "cozy", "unique", "yummy", "glad", "homemade", "best", "love", "lovely", "always", "friendly", 
                 "reasonable", "beautiful", "recommended", "fashioned", "classic", "traditional", "great", "fresh", "definitely",
                 "healthy", "decent", "generous", "comfortable", "rich", "recommend", "local", "authentic", "flavorful",
                 "enjoyed", "pleased", "flavors", "super", "happy", "absolutely", "tasty", "enjoy", "fun", "reasonably", "plenty",
                 "attentive", "truly", "fancy", "wow", "must", "nicely", "every", "quick", "easily", "early", "craving", "well"]
negativeWords = ["disappointing", "weird", "issue", "none", "last", "problem", "dirty", "mediocre", "ok", "poor", "terrible",
                 "awful", "rude", "horrible", "worst", "overpriced", "needed", "barely", "sorry", "waited", "soggy", "waiting",
                 "bland", "forgot", "hope","unfortunately", "sad", "away", "bad", "loud", "overly", "greasy", "frozen", "dry",
                 "empty", "never", "less", "hard", "expensive", "tiny", "however", "wrong", "longer", "nothing", "average", 
                 "cold", "slow", "lack", "avoid", "forget", "left", "worst"]
for i in range(0, len(names)):
    #print(i)
    #print(len(names))
    posC = 0
    negC = 0
    for word in positiveWords:
        wordCount = train[word][i]
        posC+=wordCount
    for negWord in negativeWords:
        negWordCount = train[negWord][i]
        negC+=negWordCount
    #nword = np.array(train["nword"])[i]
    nchar = np.array(train["nchar"])[i]
    #avgChars.append(nchar/nword)
    #nwords.append(nword)
    nchars.append(nchar)
    positiveCount.append(posC)
    negativeCount.append(negC)
    #posnegratio.append((posC)/(negC + 1))
    posnshort.append(posC/nword)
    negnlong.append(negC*nword)
    if posC>negC:
        positiveOverall.append(1)
    elif posC<negC:
        positiveOverall.append(-1.5)
    else:
        positiveOverall.append(0)
    #ratingArray.append(ratingAverage[names[i]])
    #numReviews.append(ratingCount[names[i]])
#predictors = np.array([positiveCount, negativeCount, ratingArray])
#predictors = np.array([nwords, positiveOverall, ratingArray, positiveCount, negativeCount, posRatio])
#posRatio = np.array(posRatio)
#print(posRatio)
predictors = np.array([positiveOverall, positiveCount, negativeCount, posnshort, negnlong, nchars])
predictors = predictors.T

#predictors=pd.DataFrame(predictors)
#newdf=[predictors, stars]
#df = pd.concat(newdf)
#df.head(5)

#Divide train/test
#x_train, x_test, y_train, y_test = train_test_split(df, stars, test_size=0.2)

#random_forest = RandomForestClassifier()
#model_rf = random_forest.fit(x_train, y_train)

#y_pred=rf.predict(x_test)

#n_folds = 5

#def rmse_cv(model):
#    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train.values)
#    rmse = np.sqrt(-cross_val_score(model, x_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
 #   return (rmse)

#score = rmse_cv(model_rf)
#print("RandomForest score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

#acc_RF= round(random_forest.score(x_test, y_test)*100,2)
#acc_RF


# smallRatingAverage = dict((k, float(ratingTotal[k])/ratingCount[k]) for k in ratingCount if ratingCount[k]<3)
# smallTotal = 0
# for k in smallRatingAverage:
#     smallTotal+=float(smallRatingAverage[k])
# print("Small Total: ", len(smallRatingAverage))
# print("Small Average: ", smallTotal/len(smallRatingAverage))

# largeRatingAverage = dict((k, float(ratingTotal[k])/ratingCount[k]) for k in ratingCount if ratingCount[k]>=50)
# largeTotal = 0
# for k in largeRatingAverage:
#     largeTotal+=float(largeRatingAverage[k])
# print("Large Total: ", len(largeRatingAverage))    
# print("Large Average: ", largeTotal/len(largeRatingAverage))


(51326, 6)

In [ ]:
testNames = test["name"]
testIDs = test["Id"]
testExpected = []
for i in range (0, len(testNames)):
    name = testNames[i]
    posC = 0
    negC = 0
    posOverall = 0
    if name in ratingAverage:
        rating = ratingAverage[name]
    else:
        rating = 3.758
        
    for word in positiveWords:
        wordCount = test[word][i]
        posC+=wordCount
    for word in negativeWords:
        wordCount = test[word][i]
        negC+=wordCount 
    if posC > negC:
        posOverall = 1
    elif posC < negC:
        posOverall = -1
    else:
        posOverall = 0
    
    nword = np.array(test["nword"])[i]
    nchar = np.array(test["nchar"])[i]
    avgChar = nchar/nword
    numReviews = ratingCount[name]
    posnshort = posC/nword
    negnlong = negC*nword
    posnegratio = ((posC)/(negC + 1))
    #predictors = np.array([[posC, negC, rating]])
    predictors = np.array([[posOverall, rating, posC, negC, posnshort, negnlong, avgChar, nword, nchar, posnegratio, numReviews]])
    prediction = random_forest.predict(predictors)
    #prediction = prediction+rating
    testExpected.append(prediction[0])
    #print(prediction)
submitDF = pd.DataFrame({'Id':testIDs, 'Expected':testExpected})
submitDF.to_csv("submission01_c.csv", sep=',', index=False)